In [1]:
import numpy as np
import pandas as pd

## #175 Combine Two Tables

In [2]:
Person = pd.DataFrame({'PersonId':[1], 'LastName':['Wang'], 'FirstName':'Allen'})
Address = pd.DataFrame({'AddressId':[1], 'PersonId':[2], 'City':['New York City'], 'State':['New York']})

In [3]:
Person

,PersonId,LastName,FirstName
0,1,Wang,Allen


In [4]:
Address

,AddressId,PersonId,City,State
0,1,2,New York City,New York


In [5]:
Person.merge(Address, how='left', on='PersonId')[['LastName','FirstName','City','State']]

,LastName,FirstName,City,State
0,Wang,Allen,NaN,NaN


## #176 Second Highest Salary

In [6]:
employee = pd.DataFrame({'Id':[1,2,3],'Salary':[100,200,300]})

In [7]:
employee

,Id,Salary
0,1,100
1,2,200
2,3,300


In [8]:
if len(employee['Salary'].unique())>=2:
    print(sorted(employee['Salary'].unique(), reverse=True)[1])
else:
    print(np.nan)

200


## #177 Nth Highest Salary

In [9]:
def getNthHighestSalary(table, n):
    if len(table['Salary'].unique()) >= n:
        return sorted(table['Salary'].unique(), reverse=True)[n-1]
    else:
        return np.nan

In [10]:
s = getNthHighestSalary(employee, 4)
s

nan

## #178 Rank Scores

In [11]:
scores = pd.DataFrame({'Id':[1,2,3,4,5,6],'Score':[3.50, 3.65, 4.00, 3.85, 4.00, 3.65]})

In [12]:
scores

,Id,Score
0,1,3.50
1,2,3.65
2,3,4.00
3,4,3.85
4,5,4.00
5,6,3.65


In [13]:
scores['Rank'] = scores['Score'].rank(axis=0,method='dense', ascending=False)

In [14]:
scores[['Score','Rank']].sort_values('Rank', ascending=True)

,Score,Rank
2,4.00,1.0
4,4.00,1.0
3,3.85,2.0
1,3.65,3.0
5,3.65,3.0
0,3.50,4.0


## #180 Consecutive numbers

In [15]:
logs = pd.DataFrame({'Id':[1,2,3,4,5,6,7],'Num':[1,1,1,2,1,2,2]})

In [16]:
logs

,Id,Num
0,1,1
1,2,1
2,3,1
3,4,2
4,5,1
5,6,2
6,7,2


In [17]:
logs['lead_1'] = logs['Num'].shift(1)
logs['lead_2'] = logs['Num'].shift(2)

In [18]:
logs[(logs['Num']==logs['lead_1']) & (logs['Num']==logs['lead_2'])]['Num'].unique()

array([1])

## #181 Employees earning more than their manager

In [19]:
employee = pd.DataFrame({'Id':[1,2,3,4], 
                         'Name':['Joe','Henry','Sam','Max'],
                         'Salary':[70000,80000,60000,90000],
                         'ManagerId':[3,4,np.nan,np.nan]})


In [20]:
employee

,Id,Name,Salary,ManagerId
0,1,Joe,70000,3.0
1,2,Henry,80000,4.0
2,3,Sam,60000,NaN
3,4,Max,90000,NaN


In [21]:
a = employee.merge(employee, how='left', left_on='ManagerId', right_on='Id')
a

,Id_x,Name_x,Salary_x,ManagerId_x,Id_y,Name_y,Salary_y,ManagerId_y
0,1,Joe,70000,3.0,3.0,Sam,60000.0,NaN
1,2,Henry,80000,4.0,4.0,Max,90000.0,NaN
2,3,Sam,60000,NaN,NaN,NaN,NaN,NaN
3,4,Max,90000,NaN,NaN,NaN,NaN,NaN


In [22]:
a[a['Salary_x']>a['Salary_y']]['Name_x'].values

array(['Joe'], dtype=object)

## #182 Duplicate emails

In [23]:
person = pd.DataFrame({'Id':[1,2,3],
                       'Email':['a@b.com','c@d.com','a@b.com']})

In [24]:
person

,Id,Email
0,1,a@b.com
1,2,c@d.com
2,3,a@b.com


In [25]:
a = person['Email'].value_counts()
a

a@b.com    2
c@d.com    1
Name: Email, dtype: int64

In [26]:
a[a>1].index[0]

'a@b.com'

## #183 Customers who never order

In [27]:
customers = pd.DataFrame({'Id':[1,2,3,4], 
                         'Name':['Joe','Henry','Sam','Max']})

In [28]:
orders = pd.DataFrame({'Id':[1,2],
                       'CustomerId':[3,1]})

In [29]:
customers

,Id,Name
0,1,Joe
1,2,Henry
2,3,Sam
3,4,Max


In [30]:
orders

,Id,CustomerId
0,1,3
1,2,1


In [31]:
a = customers.merge(orders, how='left', left_on='Id', right_on='CustomerId')
a

,Id_x,Name,Id_y,CustomerId
0,1,Joe,2.0,1.0
1,2,Henry,NaN,NaN
2,3,Sam,1.0,3.0
3,4,Max,NaN,NaN


In [32]:
a[a['Id_y'].isna()]['Name'].values

array(['Henry', 'Max'], dtype=object)

## #184 Department higest salary

In [33]:
employee = pd.DataFrame({'Id':[1,2,3,4,5], 
                         'Name':['Joe','Jim','Henry','Sam','Max'],
                         'Salary':[70000,90000,80000,60000,90000],
                         'DepartmentId':[1,1,2,2,1]})

In [34]:
department = pd.DataFrame({'Id':[1,2],
                           'Name':['IT','Sales']})

In [35]:
employee

,Id,Name,Salary,DepartmentId
0,1,Joe,70000,1
1,2,Jim,90000,1
2,3,Henry,80000,2
3,4,Sam,60000,2
4,5,Max,90000,1


In [36]:
department

,Id,Name
0,1,IT
1,2,Sales


In [37]:
employee['d_rank'] = employee.groupby('DepartmentId')['Salary'].rank('dense', ascending=False)

In [38]:
a = employee[employee['d_rank']==1]
a

,Id,Name,Salary,DepartmentId,d_rank
1,2,Jim,90000,1,1.0
2,3,Henry,80000,2,1.0
4,5,Max,90000,1,1.0


In [39]:
b = a.merge(department, how='inner', left_on='DepartmentId',right_on='Id')[['Name_x','Salary','Name_y']]
b

,Name_x,Salary,Name_y
0,Jim,90000,IT
1,Max,90000,IT
2,Henry,80000,Sales


In [40]:
b = b.rename(columns={'Name_x':'Employee', 'Name_y':'Department'})

In [41]:
b

,Employee,Salary,Department
0,Jim,90000,IT
1,Max,90000,IT
2,Henry,80000,Sales


## #185 department top 3 salaries

In [42]:
employee = pd.DataFrame({'Id':[1,2,3,4,5,6,7], 
                         'Name':['Joe','Henry','Sam','Max','Janet','Randy','Will'],
                         'Salary':[85000,80000,60000,90000,69000,85000,70000],
                         'DepartmentId':[1,2,2,1,1,1,1]})

In [43]:
employee

,Id,Name,Salary,DepartmentId
0,1,Joe,85000,1
1,2,Henry,80000,2
2,3,Sam,60000,2
3,4,Max,90000,1
4,5,Janet,69000,1
5,6,Randy,85000,1
6,7,Will,70000,1


In [44]:
department

,Id,Name
0,1,IT
1,2,Sales


In [45]:
employee['d_rank'] = employee.groupby('DepartmentId')['Salary'].rank('dense', ascending=False)

In [46]:
a = employee[employee['d_rank']<=3]
a

,Id,Name,Salary,DepartmentId,d_rank
0,1,Joe,85000,1,2.0
1,2,Henry,80000,2,1.0
2,3,Sam,60000,2,2.0
3,4,Max,90000,1,1.0
5,6,Randy,85000,1,2.0
6,7,Will,70000,1,3.0


In [47]:
b = a.merge(department, how='inner', left_on='DepartmentId',right_on='Id')[['Name_x','Salary','Name_y']]
b

,Name_x,Salary,Name_y
0,Joe,85000,IT
1,Max,90000,IT
2,Randy,85000,IT
3,Will,70000,IT
4,Henry,80000,Sales
5,Sam,60000,Sales


In [48]:
b = b.rename(columns={'Name_x':'Employee', 'Name_y':'Department'})
b

,Employee,Salary,Department
0,Joe,85000,IT
1,Max,90000,IT
2,Randy,85000,IT
3,Will,70000,IT
4,Henry,80000,Sales
5,Sam,60000,Sales


## #196 delete duplicate email

In [49]:
person = pd.DataFrame({'Id':[1,2,3], 'Email':['john@example.com', 'bob@example.com', 'john@example.com']})

In [50]:
person

,Id,Email
0,1,john@example.com
1,2,bob@example.com
2,3,john@example.com


In [51]:
person = person.sort_values('Id')

In [52]:
person = person.drop_duplicates('Email', keep='first')
person

,Id,Email
0,1,john@example.com
1,2,bob@example.com


## #197 rising temperature

In [53]:
weather = pd.DataFrame({'id':[1,2,3,4], 'recordDate':['2015-01-01', '2015-01-02', '2015-01-03', '2015-01-04'], 'temperature':[10,25,20,30]})

In [54]:
weather

,id,recordDate,temperature
0,1,2015-01-01,10
1,2,2015-01-02,25
2,3,2015-01-03,20
3,4,2015-01-04,30


In [55]:
weather = weather.sort_values('recordDate')
weather['previous_day'] = weather['recordDate'].shift(1)

In [56]:
weather['previous_temperature'] = weather['temperature'].shift(1)

In [57]:
weather['date_diff'] = pd.to_datetime(weather['recordDate']) - pd.to_datetime(weather['previous_day'])

In [58]:
weather['date_diff'] = weather['date_diff'].apply(lambda x: x.days)

In [59]:
weather[(weather['date_diff']==1) & (weather['temperature']>weather['previous_temperature'])]['id'].values

array([2, 4])

## #262 trips and users

In [60]:
trips = pd.DataFrame({'Id': [1,2,3,4,5,6,7,8,9,10],
                      'Client_Id': [1,2,3,4,1,2,3,2,3,4],
                      'Driver_Id': [10,11,12,13,10,11,12,12,10,13],
                      'City_Id': [1,1,6,6,1,6,6,12,12,12],
                      'Status': ['completed','cancelled_by_driver','completed','cancelled_by_client','completed','completed','completed','completed','completed','cancelled_by_driver'],
                      'Request_at': ['2013-10-01','2013-10-01','2013-10-01','2013-10-01','2013-10-02','2013-10-02','2013-10-02','2013-10-03','2013-10-03','2013-10-03']})

In [61]:
users = pd.DataFrame({'Users_Id': [1,2,3,4,10,11,12,13],
                      'Banned': ['No','Yes','No','No','No','No','No','No'],
                      'Role': ['client','client','client','client','driver','driver','driver','driver']})

In [62]:
trips

,Id,Client_Id,Driver_Id,City_Id,Status,Request_at
0,1,1,10,1,completed,2013-10-01
1,2,2,11,1,cancelled_by_driver,2013-10-01
2,3,3,12,6,completed,2013-10-01
3,4,4,13,6,cancelled_by_client,2013-10-01
4,5,1,10,1,completed,2013-10-02
5,6,2,11,6,completed,2013-10-02
6,7,3,12,6,completed,2013-10-02
7,8,2,12,12,completed,2013-10-03
8,9,3,10,12,completed,2013-10-03
9,10,4,13,12,cancelled_by_driver,2013-10-03


In [63]:
users

,Users_Id,Banned,Role
0,1,No,client
1,2,Yes,client
2,3,No,client
3,4,No,client
4,10,No,driver
5,11,No,driver
6,12,No,driver
7,13,No,driver


In [64]:
df = trips.merge(users[['Users_Id', 'Banned']], how='left', left_on='Client_Id', right_on='Users_Id')

In [65]:
df.rename(columns={'Banned':'Client_ban'}, inplace=True)

In [66]:
df.drop('Users_Id', inplace=True, axis=1)

In [67]:
df = df.merge(users[['Users_Id', 'Banned']], how='left', left_on='Driver_Id', right_on='Users_Id')

In [68]:
df.rename(columns={'Banned':'Driver_ban'}, inplace=True)
df.drop('Users_Id', inplace=True, axis=1)

In [69]:
df = df[(df['Client_ban']=='No') & (df['Driver_ban']=='No') & (pd.to_datetime(df['Request_at'])>=pd.to_datetime('2013-10-01')) & (pd.to_datetime(df['Request_at'])<=pd.to_datetime('2013-10-03'))]

In [70]:
df

,Id,Client_Id,Driver_Id,City_Id,Status,Request_at,Client_ban,Driver_ban
0,1,1,10,1,completed,2013-10-01,No,No
2,3,3,12,6,completed,2013-10-01,No,No
3,4,4,13,6,cancelled_by_client,2013-10-01,No,No
4,5,1,10,1,completed,2013-10-02,No,No
6,7,3,12,6,completed,2013-10-02,No,No
8,9,3,10,12,completed,2013-10-03,No,No
9,10,4,13,12,cancelled_by_driver,2013-10-03,No,No


In [71]:
df.groupby('Request_at').apply(lambda x: round(x[x['Status']!='completed'].shape[0] / x.shape[0],2))

Request_at
2013-10-01    0.33
2013-10-02    0.00
2013-10-03    0.50
dtype: float64

## #511 game play analysis 1

In [72]:
activity = pd.DataFrame({'player_id': [1,1,2,3,3],
                         'device_id': [2,2,3,1,4],
                         'event_date': ['2016-03-01', '2016-05-02', '2017-06-25', '2016-03-02', '2018-07-03'],
                         'games_played':[5,6,1,0,5]})

In [73]:
activity

,player_id,device_id,event_date,games_played
0,1,2,2016-03-01,5
1,1,2,2016-05-02,6
2,2,3,2017-06-25,1
3,3,1,2016-03-02,0
4,3,4,2018-07-03,5


In [74]:
activity.groupby('player_id')['event_date'].min()

player_id
1    2016-03-01
2    2017-06-25
3    2016-03-02
Name: event_date, dtype: object

## #512 game play analysis 2

In [75]:
activity

,player_id,device_id,event_date,games_played
0,1,2,2016-03-01,5
1,1,2,2016-05-02,6
2,2,3,2017-06-25,1
3,3,1,2016-03-02,0
4,3,4,2018-07-03,5


In [76]:
min_event_date = activity.groupby('player_id')['event_date'].min().reset_index()

In [77]:
min_event_date

,player_id,event_date
0,1,2016-03-01
1,2,2017-06-25
2,3,2016-03-02


In [78]:
activity.merge(min_event_date, how='inner', on=['player_id', 'event_date'])[['player_id', 'device_id']]

,player_id,device_id
0,1,2
1,2,3
2,3,1


## #534 game play analysis 3

In [149]:
activity = pd.DataFrame({'player_id': [1,1,1,3,3],
                         'device_id': [2,2,3,1,4],
                         'event_date': ['2016-03-01', '2016-05-02', '2017-06-25', '2016-03-02', '2018-07-03'],
                         'games_played':[5,6,1,0,5]})

In [150]:
activity

,player_id,device_id,event_date,games_played
0,1,2,2016-03-01,5
1,1,2,2016-05-02,6
2,1,3,2017-06-25,1
3,3,1,2016-03-02,0
4,3,4,2018-07-03,5


In [153]:
activity.sort_values(['player_id','event_date'], inplace=True)

In [154]:
activity

,player_id,device_id,event_date,games_played
0,1,2,2016-03-01,5
1,1,2,2016-05-02,6
2,1,3,2017-06-25,1
3,3,1,2016-03-02,0
4,3,4,2018-07-03,5


In [155]:
activity['games_played_so_far'] = activity.groupby(['player_id'])['games_played'].cumsum()

In [157]:
final_df = activity[['player_id','event_date','games_played_so_far']]

In [158]:
final_df

,player_id,event_date,games_played_so_far
0,1,2016-03-01,5
1,1,2016-05-02,11
2,1,2017-06-25,12
3,3,2016-03-02,0
4,3,2018-07-03,5


## #550 game play analysis 4

In [159]:
activity = pd.DataFrame({'player_id': [1,1,2,3,3],
                         'device_id': [2,2,3,1,4],
                         'event_date': ['2016-03-01', '2016-03-02', '2017-06-25', '2016-03-02', '2018-07-03'],
                         'games_played':[5,6,1,0,5]})

In [160]:
activity['event_date'] = pd.to_datetime(activity['event_date'])

In [161]:
activity

,player_id,device_id,event_date,games_played
0,1,2,2016-03-01,5
1,1,2,2016-03-02,6
2,2,3,2017-06-25,1
3,3,1,2016-03-02,0
4,3,4,2018-07-03,5


In [162]:
activity.sort_values(['player_id','event_date'], inplace=True)

In [167]:
activity['date_difference'] = (activity.groupby(['player_id'])['event_date'].shift(-1) - activity['event_date']).apply(lambda x: x.days)

In [171]:
activity['rank_date'] = activity.groupby(['player_id'])['event_date'].rank()

In [176]:
round(activity[(activity['date_difference']==1) & (activity['rank_date']==1)]['player_id'].nunique() / activity['player_id'].nunique(),2)

0.33

## #569 Median Employee Salary

In [181]:
employee = pd.DataFrame({'Id':np.arange(1,18),
                         'Company':['A']*6+['B']*6+['C']*5,
                         'Salary':[2341,341,15,15314,451,513,15,13,1154,1345,1221,234,2345,2645,2645,2652,65]})

In [182]:
employee

,Id,Company,Salary
0,1,A,2341
1,2,A,341
2,3,A,15
3,4,A,15314
4,5,A,451
5,6,A,513
6,7,B,15
7,8,B,13
8,9,B,1154
9,10,B,1345


In [184]:
employee.sort_values(['Company','Salary'], inplace=True)

In [186]:
employee['row_asc'] = employee.groupby('Company').cumcount()+1

In [191]:
employee['count_s'] = employee.groupby('Company')['Company'].transform('count')

In [194]:
employee['Check_median'] = employee.apply(lambda x: 'T' if x['row_asc'] in [x['count_s']/2, x['count_s']/2+1, x['count_s']/2+0.5] else 'F', axis=1)

In [196]:
final_df = employee[employee['Check_median']=='T']

In [199]:
final_df.drop(['row_asc','count_s','Check_median'], axis=1, inplace=True)

/Users/apple/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [200]:
final_df

,Id,Company,Salary
4,5,A,451
5,6,A,513
11,12,B,234
8,9,B,1154
13,14,C,2645


## #570 Managers with at Least 5 Direct Reports

In [201]:
employee = pd.DataFrame({'Id':np.arange(101, 107),
                         'Name':['John','Dan','James','Amy','Anne','Ron'],
                         'Department':['A']*5+['B'],
                         'ManagerId':[np.nan]+[101]*5})

In [202]:
employee

,Id,Name,Department,ManagerId
0,101,John,A,NaN
1,102,Dan,A,101.0
2,103,James,A,101.0
3,104,Amy,A,101.0
4,105,Anne,A,101.0
5,106,Ron,B,101.0


In [205]:
group = employee.groupby('ManagerId')['Name'].count().reset_index().rename(columns={'Name':'count'})

In [210]:
selectmanagerId = group[group['count']>=5]['ManagerId'].values

In [211]:
employee[employee['Id'].isin(selectmanagerId)]['Name']

0    John
Name: Name, dtype: object

## #571 Find Median Given Frequency of Numbers

In [212]:
numbers = pd.DataFrame({'Number':np.arange(4),
                        'Frequency':[7,1,3,1]})

In [216]:
numbers.sort_values('Number', inplace=True)

In [217]:
numbers['sum_asc'] = numbers.sort_values(['Number'])['Frequency'].cumsum()

In [219]:
numbers.sort_values(['Number'], ascending=False,inplace=True)

In [221]:
numbers['sum_desc'] = numbers['Frequency'].cumsum()

In [225]:
numbers[(numbers['sum_asc']>=sum(numbers['Frequency'])/2) & (numbers['sum_desc']>=sum(numbers['Frequency'])/2)]['Number'].mean()

0.0

## #574 Winning Candidate

In [28]:
candidate = pd.DataFrame({'id':np.arange(1,6), 'Name':['A','B','C','D','E']})

In [29]:
candidate

,id,Name
0,1,A
1,2,B
2,3,C
3,4,D
4,5,E


In [30]:
vote = pd.DataFrame({'id':np.arange(1,6), 'CandidateId':[2,4,3,2,5]})

In [31]:
vote

,id,CandidateId
0,1,2
1,2,4
2,3,3
3,4,2
4,5,5


In [35]:
group = vote.groupby('CandidateId')['id'].count().reset_index().rename(columns={'id':'count'})

In [40]:
candidate[candidate['id']==group.iloc[0]['CandidateId']]['Name']

1    B
Name: Name, dtype: object

## #577 Employee Bonus

In [41]:
employee = pd.DataFrame({'empId':np.arange(1,5),
                         'name':['John','Dan','Brad','Thomas'],
                         'supervisor':[3,3,np.nan,3],
                         'salary':[1000,2000,4000,4000]})

In [42]:
employee

,empId,name,supervisor,salary
0,1,John,3.0,1000
1,2,Dan,3.0,2000
2,3,Brad,NaN,4000
3,4,Thomas,3.0,4000


In [43]:
bonus = pd.DataFrame({'empId':[2,4],
                      'bonus':[500,2000]})

In [44]:
bonus

,empId,bonus
0,2,500
1,4,2000


In [47]:
merge_t = employee.merge(bonus, how='left', on='empId')[['name','bonus']]
merge_t[(merge_t['bonus']<1000) | pd.isnull(merge_t['bonus'])]

,name,bonus
0,John,NaN
1,Dan,500.0
2,Brad,NaN


## #578 Get Highest Answer Rate Question

In [49]:
surveylog = pd.DataFrame({'id':[5,5,5,5],
                          'action':['show','answer','show','skip'],
                          'question_id':[285,285,369,369],
                          'answer_id':[np.nan,124124,np.nan,np.nan],
                          'q_num':[1,1,2,2],
                          'timestamp':[123,124,125,126]})

In [50]:
surveylog

,id,action,question_id,answer_id,q_num,timestamp
0,5,show,285,NaN,1,123
1,5,answer,285,124124.0,1,124
2,5,show,369,NaN,2,125
3,5,skip,369,NaN,2,126


In [51]:
surveylog['show'] = surveylog['action'].apply(lambda x: 1 if x=='show' else 0)

In [53]:
surveylog['answer'] = surveylog['action'].apply(lambda x: 1 if x=='answer' else 0)

In [56]:
summary = surveylog.groupby('question_id')['show','answer'].sum().reset_index()
summary['answer_rate']=summary['answer'] / summary['show']

/Users/apple/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [58]:
summary.sort_values('answer_rate', ascending=False, inplace=True)

In [60]:
summary.sort_values('question_id', ascending=True, inplace=True)

In [63]:
summary.iloc[0]['question_id']

285.0

## #579 Find Cumulative Salary of an Employee

In [2]:
employee = pd.DataFrame({'id':[1,2,1,2,3,1,3,1,3,1,1],
                         'month':[1,1,2,2,2,3,3,4,4,7,8],
                         'salary':[20,20,30,30,40,40,60,60,70,90,90]})

In [3]:
employee

,id,month,salary
0,1,1,20
1,2,1,20
2,1,2,30
3,2,2,30
4,3,2,40
5,1,3,40
6,3,3,60
7,1,4,60
8,3,4,70
9,1,7,90


In [4]:
employee = employee.sort_values('month', ascending=True)

In [7]:
def cal_salary(single_df):
    cum_salary = np.arange(single_df.shape[0])
    for i in range(len(cum_salary)):
        mm = single_df['month'].iloc[i]
        mm_1 = mm - 1
        mm_2 = mm - 2 
        cum_salary[i] = single_df[(single_df['month']==mm) | (single_df['month']==mm_1) | (single_df['month']==mm_2)]['salary'].sum()
    single_df['cum_salary'] = cum_salary
    return single_df

In [8]:
def cal_cum_salary(df):
    new_df = []
    for _, group in df.groupby('id'):
        new_df.append(cal_salary(group))
    final_df = pd.concat(new_df)
    return final_df


In [9]:
final = cal_cum_salary(employee)

In [10]:
final['rnk'] = final.groupby('id')['month'].rank(ascending=False)

In [11]:
final[final['rnk']!=1].drop('rnk', axis=1)

,id,month,salary,cum_salary
0,1,1,20,20
2,1,2,30,50
5,1,3,40,90
7,1,4,60,130
9,1,7,90,90
1,2,1,20,20
4,3,2,40,40
6,3,3,60,100


## #580 Count Student Number in Departments

In [23]:
student = pd.DataFrame({'student_id':[1,2,3],
                        'student_name':['Jack','Jane','Mark'],
                        'gender':['M','F','M'],
                        'dept_id':[1,1,2]})

In [24]:
student

,student_id,student_name,gender,dept_id
0,1,Jack,M,1
1,2,Jane,F,1
2,3,Mark,M,2


In [25]:
department = pd.DataFrame({'dept_id':[1,2,3],
                           'dept_name':['Engineering','Science','Law']})

In [27]:
department

,dept_id,dept_name
0,1,Engineering
1,2,Science
2,3,Law
